In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [2]:
authors = []
dates = []
statements = []
sources = []
targets = []

In [3]:
def scrape_website(page_number):
  page_num = str(page_number)
  URL = 'https://www.politifact.com/factchecks/list/?page='+page_num
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text, 'html.parser')
  statement_footer = soup.find_all ('footer' , attrs={'class' : 'm-statement__footer'})
  statement_quote = soup.find_all ('div' , attrs={'class' : 'm-statement__quote'})
  statement_meta = soup.find_all ('div' , attrs={'class' : 'm-statement__meta'})
  target = soup.find_all ('div' , attrs={'class' : 'm-statement__meter'})

  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date= link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name + ' ' + last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month+' '+day+' '+year
    dates.append(date)
    authors.append(full_name)   

  for i in statement_quote:
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    statements.append(statement_text)

  for i in statement_meta:
    link3 = i.find_all('a')
    source_text = link3[0].text.strip()
    sources.append(source_text)

  for i in target:
    link4 = i.find('div' , attrs={'class' : 'c-image'}).find('img').get('alt')
    targets.append(link4)

In [4]:
n = 2
for i in range(1, n):
  scrape_website(i)

In [7]:
data = pd.DataFrame(columns = ['author','statement','source','date','target'])
data['author']= authors
data['statement']= statements
data['source']= sources
data['date']= dates
data['target']= targets


In [8]:
data

,author,statement,source,date,target
0,Jon Greenberg,Masks have negative impacts on 68% of children.,Instagram posts,"January 29, 2021",false
1,Samantha Putterman,Says Joe Biden’s inauguration was fake and he ...,Facebook posts,"January 29, 2021",pants-fire
2,Tom Kertscher,"Says Joe Biden said, “I don’t know what I’m si...",Facebook posts,"January 29, 2021",false
3,Tom Kertscher,“U.S. military at the White House arresting Co...,Facebook posts,"January 29, 2021",pants-fire
4,Miriam Valverde,"""Only in the Third World do you see this habit...",Marco Rubio,"January 29, 2021",false
5,Brandon Mulder,Says that Biden’s ban on new federal leases wi...,Kevin Brady,"January 29, 2021",half-true
6,Jon Greenberg,“84% of all new electric capacity planned to c...,Joe Biden,"January 29, 2021",mostly-true
7,Tom Kertscher,Says Joe Biden canceled the Keystone pipeline ...,Facebook posts,"January 29, 2021",false
8,Madison Czopek,"""In March 2020 the United States deployed 30,0...",Facebook posts,"January 29, 2021",false
9,Daniel Funke,Says Joe Biden is “following through with a fr...,Facebook posts,"January 28, 2021",barely-true


In [11]:
def getBinaryNumTarget(text):
   if text == 'true':
     return 1
   else:
     return 0
      

In [12]:

def getBinaryTarget(text):
   if text == 'true':
     return 'REAL'
   else:
     return 'FAKE'

In [13]:
#creating the two columns on the dataframe
data['BinaryTarget'] = data['target'].apply(getBinaryTarget)
data['BinaryNumTarget'] = data['target'].apply(getBinaryNumTarget)

In [15]:
#show the data
data


,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
0,Jon Greenberg,Masks have negative impacts on 68% of children.,Instagram posts,"January 29, 2021",false,FAKE,0
1,Samantha Putterman,Says Joe Biden’s inauguration was fake and he ...,Facebook posts,"January 29, 2021",pants-fire,FAKE,0
2,Tom Kertscher,"Says Joe Biden said, “I don’t know what I’m si...",Facebook posts,"January 29, 2021",false,FAKE,0
3,Tom Kertscher,“U.S. military at the White House arresting Co...,Facebook posts,"January 29, 2021",pants-fire,FAKE,0
4,Miriam Valverde,"""Only in the Third World do you see this habit...",Marco Rubio,"January 29, 2021",false,FAKE,0
5,Brandon Mulder,Says that Biden’s ban on new federal leases wi...,Kevin Brady,"January 29, 2021",half-true,FAKE,0
6,Jon Greenberg,“84% of all new electric capacity planned to c...,Joe Biden,"January 29, 2021",mostly-true,FAKE,0
7,Tom Kertscher,Says Joe Biden canceled the Keystone pipeline ...,Facebook posts,"January 29, 2021",false,FAKE,0
8,Madison Czopek,"""In March 2020 the United States deployed 30,0...",Facebook posts,"January 29, 2021",false,FAKE,0
9,Daniel Funke,Says Joe Biden is “following through with a fr...,Facebook posts,"January 28, 2021",barely-true,FAKE,0
